# Auto Polling
    - The original persona generation worked, but the personas were too generic and didnt give a model context to 
        make accurate predictions of the beliefs of a person
    - Now making a new pipeline that will create personas in different stages to 
        

In [1]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string
import random
import numpy as np
import pickle

In [2]:
def create_directory_if_not_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load First and Last Names

In [3]:
eths = ["White", "American Indian or Alaska Native", "Black or African American", "Asian", "Hispanic or Latino"]
genders = ['Men', 'Women']

dems = []
for e in eths:
    for g in genders:
        s = f"{e} {g}"
        dems.append(s)
        
first_names = {}
for d in dems:
    # Open the file in read binary mode
    with open(f"Names/First_Names/{d}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        first_names[d] = my_list

last_names = {}
for e in eths:
    # Open the file in read binary mode
    with open(f"Names/Last_Names/{e}.pkl", "rb") as file:
        # Use pickle to load the list object from the file
        my_list = pickle.load(file)
        last_names[e] = my_list

# Distributing Statistics

In [4]:
def distribute_geographic_location():
    random_number = random.random()
    
    if random_number < 0.44:
        return "Rural Montana, United States"
    else:
        return "Urban Montana, United States"
    
def distribute_gender():
    random_number = random.random()
    
    if random_number < 0.49:
        return "Female"
    else:
        return "Male"
    
def distribute_ethnicity():
    random_number = random.random()
    
    if random_number < 0.88:
        return "White"
    elif random_number < 0.94:
        return "American Indian or Alaska Native"
    elif random_number < 0.95:
        return "Black or African American"
    elif random_number < 0.96:
        return "Asian"
    else:
        return "Hispanic or Latino"
    
def distribute_age_and_income():
    random_number = random.random()
    
    if random_number < 0.13:
        age = random.randint(18, 25)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 32413  # Mean of the distribution
        std_dev = 10000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.44:
        age = random.randint(25, 44)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 69192  # Mean of the distribution
        std_dev = 20000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    elif random_number < 0.75:
        age = random.randint(45, 64)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 71064  # Mean of the distribution
        std_dev = 25000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    else:
        age = random.randint(65, 87)
        
        # Set the mean and standard deviation of the normal distribution
        mean = 47015  # Mean of the distribution
        std_dev = 15000  # Standard deviation of the distribution
        income = round(int(np.random.normal(mean, std_dev, size=1)[0]), -2)
        
        return age, income
    
def distribute_education_level():
    random_number = random.random()
    
    if random_number < 0.60:
        return "High school graduate"
    elif random_number < 0.94:
        return "Bachelor's Degree or Higher"
    else:
        return "Less than a high school education"
    
def distribute_party():
    """
    # Distribute Politcal Affiliation
    - Reps that say they are conservative at Con
    - Reps say moderate, right leaning
    - Anyone that contradicts or says dont know is moderate
    - Dems that say moderate, left leaning
    - Dems that say liberal, liberal
    """
    random_number = random.random()
    
    if random_number < 0.42:
        return "Conservative"
    elif random_number < 0.56:
        return "Right Leaning"
    elif random_number < 0.66:
        return "Moderate"
    elif random_number < 0.81:
        return "Left Leaning"
    else:
        return "Liberal"
    
def distribute_veteran():
    random_number = random.random()
    
    if random_number < 0.9:
        return "Not a Veteran"
    else:
        return "Is a Veteran"

# Generate Names

In [26]:
def generate_name(eth, gend):
    if gend == 'Male':
        gen = 'Men'
    else:
        gen = 'Women'
    f_names = first_names[f"{eth} {gen}"]
    
    first = f_names[random.randint(0,len(f_names)-1)].strip()
    
    l_names = last_names[f"{eth}"]
    last = l_names[random.randint(0,len(l_names)-1)].strip()
    
    return f"{first} {last}"

In [27]:
def make_persona():
    p = {}
    
    # Distribute Ethnicity
    eth = distribute_ethnicity()
    # Distribute Gender
    gend = distribute_gender()
    
    p['Name'] = generate_name(eth, gend)
    
    # Distribute Age and Income
    age, income = distribute_age_and_income()
    p['Age'] = str(age)
    p['Ethnicity'] = eth
    p['Gender'] = gend
    
    p['Income'] = str(income)

    # Distribute Education Level
    p['Education Level'] = distribute_education_level()
    
    # Distribute Political Affiliation
    p['Political Affiliation'] = distribute_party()
    
    # Distribute Geographic Location
    p['Geographic Location'] = distribute_geographic_location()
    
    # Distribute Veteran Status
    p['Veteran Status'] = distribute_veteran()
    
    return p

In [28]:
poll_size = 100
pp = []
for i in range(poll_size):
    pp.append(make_persona())

In [33]:
TOKENS = 0

In [34]:
def gen_person_occupation(p):
    occ_gen_role = GPT.open_file("Prompts/OccupationGen/occ_gen_role.txt")
    prompt = GPT.open_file("Prompts/OccupationGen/occ_gen_prompt.txt").replace('<<PERSON>>', str(p))
    occ, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 50, role=occ_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Occupation'] = occ
    
    return p

In [35]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_occ = list(executor.map(gen_person_occupation, pp))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID afc783503026b6b45fd939ee0e3e28e9 in your message.)


In [36]:
def gen_person_media(p):
    media_gen_role = GPT.open_file("Prompts/MediaGen/media_gen_role.txt")
    prompt = GPT.open_file("Prompts/MediaGen/media_gen_in.txt").replace('<<PERSON>>', str(p))
    media, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=media_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Media Consumption'] = media
    
    return p

In [37]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_media = list(executor.map(gen_person_media, pp_occ))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a29ea600c4fe8593c5a61fddd2be3058 in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a2aa0649983bca9dd7a692742f46513c in your message.)


In [38]:
def gen_person_voting_history(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_voting_history.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Voting History'] = vh
    
    return p

In [39]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_vote = list(executor.map(gen_person_voting_history, pp_media))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 99340a5ec629c7e9f0cabaa22b3654b4 in your message.)


In [40]:
def gen_person_group_membership(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_group_membership.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Group Membership'] = vh
    
    return p

In [41]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_mem = list(executor.map(gen_person_group_membership, pp_vote))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1aa54845c6e104ae274f885c8ee9fb0e in your message.)
Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 287d48cd72effe31e566cb0cb82aa563 in your message.)


In [42]:
def gen_person_group_membership(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_influential_figures.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Influential Figures'] = vh
    
    return p

In [43]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_if = list(executor.map(gen_person_group_membership, pp_mem))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID afe405028de22c6d47e7784299cd3253 in your message.)


In [44]:
def gen_person_personal_anecdotes(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/personal_anecdotes.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Personal Anecdotes'] = vh
    
    return p

In [45]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_anec = list(executor.map(gen_person_personal_anecdotes, pp_if))

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID f6d3e03cd0661f8796a096d966a88385 in your message.)
Error communicating with OpenAI: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Fri, 19 May 2023 18:07:42 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7c9e4028f8fa81bb-IAD', 'alt-svc': 'h3=":443"; ma=86400, h3-29=":443"; ma=86400'}


In [46]:
def gen_person_future(p):
    voting_gen_role = GPT.open_file("Prompts/AskingPersona/load_persona_role.txt").replace('<<PERSON>>', str(p))
    prompt = GPT.open_file("Prompts/AskingPersona/gen_future_aspirations_and_concerns.txt")
    vh, usage = GPT.chat_gpt(prompt, engine='gpt-3.5-turbo', temp=0.1, tokens = 400, role=voting_gen_role)
    
    global TOKENS
    TOKENS += usage['total_tokens']
    
    p['Future Aspirations/Concerns'] = vh
    
    return p

In [47]:
# Create a thread pool with 10 workers
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit each person to the worker pool
    pp_future = list(executor.map(gen_person_future, pp_anec))

In [48]:
TOKENS

422011

In [49]:
cost = TOKENS/1000*0.002
print(cost)

0.844022


In [52]:
pp_future[3]

{'Name': 'Kevin Evans',
 'Age': '64',
 'Ethnicity': 'White',
 'Gender': 'Male',
 'Income': '86900',
 'Education Level': 'High school graduate',
 'Political Affiliation': 'Right Leaning',
 'Geographic Location': 'Urban Montana, United States',
 'Veteran Status': 'Not a Veteran',
 'Occupation': 'Retired Business Owner.',
 'Media Consumption': 'Based on the demographic information provided, Kevin Evans is likely to trust and consume media outlets that cater to a right-leaning, older, white, male audience. He may also be interested in news and information related to business and finance.\n\nSome media outlets that Kevin Evans may trust and consume regularly include:\n\n- Fox News\n- The Wall Street Journal\n- The Drudge Report\n- Breitbart News\n- The Blaze\n\nAs a retired business owner, Kevin Evans may have more free time to consume media than someone who is still working. However, as an older individual, he may not be as tech-savvy as younger generations and may prefer traditional forms

In [51]:
import json
file_path = "new_personas_100.json"

# Open the file in write mode and save the dictionary as JSON
with open(file_path, "w") as json_file:
    json.dump(pp_future, json_file)

In [28]:
def dict_to_paragraph(persona, residence):
    priorities = ', '.join(persona["Top Priorities"])
    try:
        pain_points = ', '.join(persona["Pain points"])
    except:
        pain_points = ', '.join(persona["Pain Points"])
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}. "
            f"You live in {residence} and your annual income is ${persona['Income']}. "
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}. "
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'. "
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.")
